In [4]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import re

In [9]:
HOST = 'https://www.1111.com.tw'
HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36'}

# global variable
thread_list=[]
alinks = []
wc = Counter()
lock = threading.Lock()
link_path = '../data/link_1111.csv'

#Get total pages
url =  HOST + '/job-bank/job-index.asp?si=1&d0=140400,140200,140300&fs=1&ps=100&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
page = int(soup.select_one('div.pagedata').text.split('/')[1].split('頁')[0])


# Get all links from all pages
for i in range(1,page):
    url =  HOST + '/job-bank/job-index.asp?si=1&d0=140400,140200,140300&fs=1&ps=100&page={}'.format(i)
    th = threading.Thread(target=get_pagelinks, args=(url, lock, i, link_path))
    th.start()
    time.sleep(1)
    thread_list.append(th)
else: 
    print('[Debug] All threads are started')

# Join all threads
for thread in thread_list:
    thread.join()

thread_list2 = []

# Not applicable to this case !!
# Get all content from alllinks
# with ThreadPoolExecutor(max_workers=5) as executor:
#     for idx, link in enumerate(alinks):
#         executor.submit(grab_content, link, lock, idx) 

for idx, link in enumerate(alinks):
    th = threading.Thread(target=grab_content, args=(link, lock, idx ))
    th.start()
    time.sleep(0.2)
    thread_list2.append(th)
for thread in thread_list2:
    thread.join()
        
# Wirte results to file
with open('../data_final/1111.csv', 'w') as fw2:
    for lang, counts in wc.most_common():
        fw2.write('{},{}\n'.format(lang,counts))

0 have been completed
Cannot retrieve contentfrom page 28 Retry: 1 
Cannot retrieve contentfrom page 27 Retry: 1 
Cannot retrieve contentfrom page 28 Retry: 2 Cannot retrieve contentfrom page 27 Retry: 2 

Cannot retrieve contentfrom page 28 Retry: 3 
Cannot retrieve contentfrom page 27 Retry: 3 
Cannot retrieve contentfrom page 28 Retry: 4 
Cannot retrieve contentfrom page 27 Retry: 4 
Cannot retrieve contentfrom page 27 Retry: 5 Cannot retrieve contentfrom page 28 Retry: 5 

200 have been completed
Cannot retrieve contentfrom page 228 Retry: 1 
Cannot retrieve contentfrom page 228 Retry: 2 
Cannot retrieve contentfrom page 228 Retry: 3 
Cannot retrieve contentfrom page 228 Retry: 4 
Cannot retrieve contentfrom page 228 Retry: 5 
400 have been completed
Cannot retrieve contentfrom page 482 Retry: 1 
Cannot retrieve contentfrom page 482 Retry: 2 
Cannot retrieve contentfrom page 482 Retry: 3 
Cannot retrieve contentfrom page 482 Retry: 4 
Cannot retrieve contentfrom page 482 Retry: 5 


In [12]:
def get_pagelinks(url, lock, i, link_path):  #找所有連結
    
    global HEADER
    global alinks
    global total_links
    count = 5             # retry 5 times
    while count:
        try:
            count = 5             # retry 5 times
            while count:
                res = requests.get(url, headers=HEADER)
                if res.status_code == 200 and len(links) != 0:

                    soup = BeautifulSoup(res.text, 'lxml')
                    links = soup.select('div.jbInfo > div > h3 > a')

                    for link in links:  
                        page_links.append('https:' + str(link['href'])) #href put in innerURL function           
                    print('Get {} links from page {}'.format(len(page_links), i))

                    with open(link_path, 'a') as fw:
                        with lock:
                            alinks += page_links
                            fw.write('\n'.join(page_links))
                            fw.write('\n')
                    break
                else:
                    print ('Cannot retrieve links from page {} Retry: {} '.format(i, 6 - count))
                    count -= 1
                    time.sleep(1)
        except Exception as e:
            print ('Cannot retrieve links from page {} Retry: {} Error: {}'.format(i, 6 - count, e))
            count -= 1
            time.sleep(1)
            
    if count == 0:
        print('[Error] Page {:2} failed'.format(i))

In [11]:
def grab_content(url,lock,idx): #一個用來分析內頁的function
    
    global total_links
    global HEADER
    global wc

    count = 5
    while count:
        try:
            res2 = requests.get(url, headers=HEADER)
            if res2.status_code == 200:
                soup = BeautifulSoup(res2.text, 'lxml')
                dds = soup.select('dl.dataList > dd')
                text = ''
                for dd in dds:
                    for a in dd.find_all('a'):
                        a.decompose()
                    for img in dd.find_all('img'):
                        img.decompose()
                    text += dd.text
                words = list(set(re.findall('object c|visual basic|[A-Za-z.+#]+', text, re.IGNORECASE)))
                with lock:      
                    for lan in words: 
                        if lan in wc:  
                            wc[lan] += 1
                        else:
                            wc[lan] = 1
                if idx % 200 == 0:
                    print('{} have been completed'.format(idx))
                break
            else:
                print ('Cannot retrieve contentfrom page {} Retry: {} '.format(idx, 6 - count))
                count -= 1
                time.sleep(0.5)
        except Exception as e:
            print ('Cannot retrieve contentfrom page {} Retry: {} Error; {} '.format(idx, 6 - count), e)
            count -= 1
            time.sleep(0.5)
    if count == 0:
        print('Retrieve contetn failed page {} '.format(idx))

In [10]:
wc.most_common()

[('.', 5456),
 ('C', 1366),
 ('SQL', 1353),
 ('C++', 1175),
 ('Java', 1127),
 ('MIS', 1116),
 ('MS', 868),
 ('Windows', 847),
 ('Linux', 799),
 ('Excel', 788),
 ('and', 778),
 ('Word', 766),
 ('PM', 757),
 ('HTML', 755),
 ('C#', 732),
 ('JAVA', 726),
 ('AM', 691),
 ('Server', 686),
 ('PowerPoint', 626),
 ('Script', 625),
 ('with', 577),
 ('ASP.NET', 561),
 ('Android', 552),
 ('or', 537),
 ('Web', 527),
 ('CSS', 515),
 ('MySQL', 514),
 ('in', 497),
 ('Oracle', 479),
 ('to', 473),
 ('PHP', 468),
 ('of', 464),
 ('LINUX', 450),
 ('C#.NET', 438),
 ('for', 435),
 ('ERP', 433),
 ('Outlook', 428),
 ('jQuery', 412),
 ('a', 396),
 ('development', 387),
 ('the', 385),
 ('APP', 367),
 ('experience', 361),
 ('is', 329),
 ('JavaScript', 329),
 ('IP', 324),
 ('on', 319),
 ('Visual', 317),
 ('MVC', 308),
 ('system', 295),
 ('Python', 293),
 ('TCP', 279),
 ('HR', 272),
 ('design', 265),
 ('API', 264),
 ('AJAX', 260),
 ('iOS', 249),
 ('IT', 240),
 ('Javascript', 240),
 ('mail', 231),
 ('software', 228),